<div style="text-align: center;">
    <h1> <font style="bold"> Trabajo Práctico </font></h1>
    <h2><font style="bold">LDA/QDA y optimización matemática de modelos</font></h2>
    <h3><font style="bold">Abril Noguera - Pablo Brahim - Fermin Rodriguez - Kevin Pennington</font></h3>
</div>

# Intro teórica

## Definición: Clasificador Bayesiano

Sean $k$ poblaciones, $x \in \mathbb{R}^p$ puede pertenecer a cualquiera $g \in \mathcal{G}$ de ellas. Bajo un esquema bayesiano, se define entonces $\pi_j \doteq P(G = j)$ la probabilidad *a priori* de que $X$ pertenezca a la clase *j*, y se **asume conocida** la distribución condicional de cada observable dado su clase $f_j \doteq f_{X|G=j}$.

De esta manera dicha probabilidad *a posteriori* resulta
$$
P(G|_{X=x} = j) = \frac{f_{X|G=j}(x) \cdot p_G(j)}{f_X(x)} \propto f_j(x) \cdot \pi_j
$$

La regla de decisión de Bayes es entonces
$$
H(x) \doteq \arg \max_{g \in \mathcal{G}} \{ P(G|_{X=x} = j) \} = \arg \max_{g \in \mathcal{G}} \{ f_j(x) \cdot \pi_j \}
$$

es decir, se predice a $x$ como perteneciente a la población $j$ cuya probabilidad a posteriori es máxima.

*Ojo, a no desesperar! $\pi_j$ no es otra cosa que una constante prefijada, y $f_j$ es, en su esencia, un campo escalar de $x$ a simplemente evaluar.*

## Distribución condicional

Para los clasificadores de discriminante cuadrático y lineal (QDA/LDA) se asume que $X|_{G=j} \sim \mathcal{N}_p(\mu_j, \Sigma_j)$, es decir, se asume que cada población sigue una distribución normal.

Por definición, se tiene entonces que para una clase $j$:
$$
f_j(x) = \frac{1}{(2 \pi)^\frac{p}{2} \cdot |\Sigma_j|^\frac{1}{2}} e^{- \frac{1}{2}(x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j)}
$$

Aplicando logaritmo (que al ser una función estrictamente creciente no afecta el cálculo de máximos/mínimos), queda algo mucho más práctico de trabajar:

$$
\log{f_j(x)} = -\frac{1}{2}\log |\Sigma_j| - \frac{1}{2} (x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j) + C
$$

Observar que en este caso $C=-\frac{p}{2} \log(2\pi)$, pero no se tiene en cuenta ya que al tener una constante aditiva en todas las clases, no afecta al cálculo del máximo.

## LDA

En el caso de LDA se hace una suposición extra, que es $X|_{G=j} \sim \mathcal{N}_p(\mu_j, \Sigma)$, es decir que las poblaciones no sólo siguen una distribución normal sino que son de igual matriz de covarianzas. Reemplazando arriba se obtiene entonces:

$$
\log{f_j(x)} =  -\frac{1}{2}\log |\Sigma| - \frac{1}{2} (x-\mu_j)^T \Sigma^{-1} (x- \mu_j) + C
$$

Ahora, como $-\frac{1}{2}\log |\Sigma|$ es común a todas las clases se puede incorporar a la constante aditiva y, distribuyendo y reagrupando términos sobre $(x-\mu_j)^T \Sigma^{-1} (x- \mu_j)$ se obtiene finalmente:

$$
\log{f_j(x)} =  \mu_j^T \Sigma^{-1} (x- \frac{1}{2} \mu_j) + C'
$$

## Entrenamiento/Ajuste

Obsérvese que para ambos modelos, ajustarlos a los datos implica estimar los parámetros $(\mu_j, \Sigma_j) \; \forall j = 1, \dots, k$ en el caso de QDA, y $(\mu_j, \Sigma)$ para LDA.

Estos parámetros se estiman por máxima verosimilitud, de manera que los estimadores resultan:

* $\hat{\mu}_j = \bar{x}_j$ el promedio de los $x$ de la clase *j*
* $\hat{\Sigma}_j = s^2_j$ la matriz de covarianzas estimada para cada clase *j*
* $\hat{\pi}_j = f_{R_j} = \frac{n_j}{n}$ la frecuencia relativa de la clase *j* en la muestra
* $\hat{\Sigma} = \frac{1}{n} \sum_{j=1}^k n_j \cdot s^2_j$ el promedio ponderado (por frecs. relativas) de las matrices de covarianzas de todas las clases. *Observar que se utiliza el estimador de MV y no el insesgado*

Es importante notar que si bien todos los $\mu, \Sigma$ deben ser estimados, la distribución *a priori* puede no inferirse de los datos sino asumirse previamente, utilizándose como entrada del modelo.

## Predicción

Para estos modelos, al igual que para cualquier clasificador Bayesiano del tipo antes visto, la estimación de la clase es por método *plug-in* sobre la regla de decisión $H(x)$, es decir devolver la clase que maximiza $\hat{f}_j(x) \cdot \hat{\pi}_j$, o lo que es lo mismo $\log\hat{f}_j(x) + \log\hat{\pi}_j$.

# Código provisto

Con el fin de no retrasar al alumno con cuestiones estructurales y/o secundarias al tema que se pretende tratar, se provee una base de código que **no es obligatoria de usar** pero se asume que resulta resulta beneficiosa.

In [1]:
import numpy as np
import pandas as pd
import numpy.linalg as LA
from scipy.linalg import cholesky, solve_triangular
from scipy.linalg.lapack import dtrtri

## Base code

In [3]:
class BaseBayesianClassifier:
  def __init__(self):
    pass

  def _estimate_a_priori(self, y):
    a_priori = np.bincount(y.flatten().astype(int)) / y.size
    # Q3: para que sirve bincount? --> cuenta cuantas veces aparece cada clase en y.
    return np.log(a_priori)

  def _fit_params(self, X, y):
    # estimate all needed parameters for given model
    raise NotImplementedError()

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    raise NotImplementedError()

  def fit(self, X, y, a_priori=None):
    # if it's needed, estimate a priori probabilities
    self.log_a_priori = self._estimate_a_priori(y) if a_priori is None else np.log(a_priori)

    # now that everything else is in place, estimate all needed parameters for given model
    self._fit_params(X, y)
    # Q4: por que el _fit_params va al final? no se puede mover a, por ejemplo, antes de la priori? --> 

  def predict(self, X):
    # this is actually an individual prediction encased in a for-loop
    m_obs = X.shape[1]
    y_hat = np.empty(m_obs, dtype=int)

    for i in range(m_obs):
      y_hat[i] = self._predict_one(X[:,i].reshape(-1,1))

    # return prediction as a row vector (matching y)
    return y_hat.reshape(1,-1)

  def _predict_one(self, x):
    # calculate all log posteriori probabilities (actually, +C)
    log_posteriori = [ log_a_priori_i + self._predict_log_conditional(x, idx) for idx, log_a_priori_i
                  in enumerate(self.log_a_priori) ]

    # return the class that has maximum a posteriori probability
    return np.argmax(log_posteriori)

In [5]:
class QDA(BaseBayesianClassifier):

  def _fit_params(self, X, y):
    # estimate each covariance matrix
    self.inv_covs = [LA.inv(np.cov(X[:,y.flatten()==idx], bias=True))
                      for idx in range(len(self.log_a_priori))]
    # Q5: por que hace falta el flatten y no se puede directamente X[:,y==idx]?
    # Q6: por que se usa bias=True en vez del default bias=False?
    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]
    # Q7: que hace axis=1? por que no axis=0?

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    inv_cov = self.inv_covs[class_idx]
    unbiased_x =  x - self.means[class_idx]
    return 0.5*np.log(LA.det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x

In [6]:
class TensorizedQDA(QDA):

    def _fit_params(self, X, y):
        # ask plain QDA to fit params
        super()._fit_params(X,y)

        # stack onto new dimension
        self.tensor_inv_cov = np.stack(self.inv_covs) # → shape: (k, p, p)
        self.tensor_means = np.stack(self.means) # → shape: (k, p, 1)

    def _predict_log_conditionals(self,x):
        unbiased_x = x - self.tensor_means # → shape: (k, p, 1)
        inner_prod = unbiased_x.transpose(0,2,1) @ self.tensor_inv_cov @ unbiased_x # → shape: (k, 1, 1)

        return 0.5*np.log(LA.det(self.tensor_inv_cov)) - 0.5 * inner_prod.flatten() # → shape: (k, )

    def _predict_one(self, x):
        # return the class that has maximum a posteriori probability
        return np.argmax(self.log_a_priori + self._predict_log_conditionals(x))

In [7]:
class QDA_Chol1(BaseBayesianClassifier):
  def _fit_params(self, X, y):
    self.L_invs = [
        LA.inv(cholesky(np.cov(X[:,y.flatten()==idx], bias=True), lower=True))
        for idx in range(len(self.log_a_priori))
    ]

    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]

  def _predict_log_conditional(self, x, class_idx):
    L_inv = self.L_invs[class_idx]
    unbiased_x =  x - self.means[class_idx]

    y = L_inv @ unbiased_x

    return np.log(L_inv.diagonal().prod()) -0.5 * (y**2).sum()

In [8]:
class QDA_Chol2(BaseBayesianClassifier):
  def _fit_params(self, X, y):
    self.Ls = [
        cholesky(np.cov(X[:,y.flatten()==idx], bias=True), lower=True)
        for idx in range(len(self.log_a_priori))
    ]

    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]

  def _predict_log_conditional(self, x, class_idx):
    L = self.Ls[class_idx]
    unbiased_x =  x - self.means[class_idx]

    y = solve_triangular(L, unbiased_x, lower=True)

    return -np.log(L.diagonal().prod()) -0.5 * (y**2).sum()

In [9]:
class QDA_Chol3(BaseBayesianClassifier):
  def _fit_params(self, X, y):
    self.L_invs = [
        dtrtri(cholesky(np.cov(X[:,y.flatten()==idx], bias=True), lower=True), lower=1)[0]
        for idx in range(len(self.log_a_priori))
    ]

    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]

  def _predict_log_conditional(self, x, class_idx):
    L_inv = self.L_invs[class_idx]
    unbiased_x =  x - self.means[class_idx]

    y = L_inv @ unbiased_x

    return np.log(L_inv.diagonal().prod()) -0.5 * (y**2).sum()

## Datasets

In [10]:
from sklearn.datasets import load_iris, fetch_openml, load_wine
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

def get_iris_dataset():
  data = load_iris()
  X_full = data.data
  y_full = np.array([data.target_names[y] for y in data.target.reshape(-1,1)])
  return X_full, y_full

def get_penguins_dataset():
    # get data
    df, tgt = fetch_openml(name="penguins", return_X_y=True, as_frame=True, parser='auto')

    # drop non-numeric columns
    df.drop(columns=["island","sex"], inplace=True)

    # drop rows with missing values
    mask = df.isna().sum(axis=1) == 0
    df = df[mask]
    tgt = tgt[mask]

    return df.values, tgt.to_numpy().reshape(-1,1)

def get_wine_dataset():
    # get data
    data = load_wine()
    X_full = data.data
    y_full = np.array([data.target_names[y] for y in data.target.reshape(-1,1)])
    return X_full, y_full

def get_letters_dataset():
    # get data
    letter = fetch_openml('letter', version=1, as_frame=False)
    return letter.data, letter.target.reshape(-1,1)

def label_encode(y_full):
    return LabelEncoder().fit_transform(y_full.flatten()).reshape(y_full.shape)

def split_transpose(X, y, test_size, random_state):
    # X_train, X_test, y_train, y_test but all transposed
    return [elem.T for elem in train_test_split(X, y, test_size=test_size, random_state=random_state)]

## Benchmarking

Nota: esta clase fue creada bastante rápido y no pretende ser una plataforma súper confiable sobre la que basarse, sino más bien una herramienta simple con la que poder medir varios runs y agregar la información.

En forma rápida, `warmup` es la cantidad de runs para warmup, `mem_runs` es la cantidad de runs en las que se mide el pico de uso de RAM y `n_runs` es la cantidad de runs en las que se miden tiempos.

La razón por la que se separan es que medir memoria hace ~2.5x más lento cada run, pero al mismo tiempo se estabiliza mucho más rápido.

**Importante:** tener en cuenta que los modelos que predicen en batch (usan `predict` directamente) deberían consumir, como mínimo, $n$ veces la memoria de los que predicen por observación.

In [11]:
import time
from tqdm import tqdm
from numpy.random import RandomState
import tracemalloc

RNG_SEED = 6553

class Benchmark:
    def __init__(self, X, y, n_runs=1000, warmup=100, mem_runs=100, test_sz=0.3, rng_seed=RNG_SEED, same_splits=True):
        self.X = X
        self.y = y
        self.n = n_runs
        self.warmup = warmup
        self.mem_runs = mem_runs
        self.test_sz = test_sz
        self.det = same_splits
        if self.det:
            self.rng_seed = rng_seed
        else:
            self.rng = RandomState(rng_seed)

        self.data = dict()

        print("Benching params:")
        print("Total runs:",self.warmup+self.mem_runs+self.n)
        print("Warmup runs:",self.warmup)
        print("Peak Memory usage runs:", self.mem_runs)
        print("Running time runs:", self.n)
        approx_test_sz = int(self.y.size * self.test_sz)
        print("Train size rows (approx):",self.y.size - approx_test_sz)
        print("Test size rows (approx):",approx_test_sz)
        print("Test size fraction:",self.test_sz)

    def bench(self, model_class, **kwargs):
        name = model_class.__name__
        time_data = np.empty((self.n, 3), dtype=float)  # train_time, test_time, accuracy
        mem_data = np.empty((self.mem_runs, 2), dtype=float)  # train_peak_mem, test_peak_mem
        rng = RandomState(self.rng_seed) if self.det else self.rng


        for i in range(self.warmup):
            # Instantiate model with error check for unsupported parameters
            model = model_class(**kwargs)

            # Generate current train-test split
            X_train, X_test, y_train, y_test = split_transpose(
                self.X, self.y,
                test_size=self.test_sz,
                random_state=rng
            )
            # Run training and prediction (timing or memory measurement not recorded)
            model.fit(X_train, y_train)
            model.predict(X_test)

        for i in tqdm(range(self.mem_runs), total=self.mem_runs, desc=f"{name} (MEM)"):

            model = model_class(**kwargs)

            X_train, X_test, y_train, y_test = split_transpose(
                self.X, self.y,
                test_size=self.test_sz,
                random_state=rng
            )

            tracemalloc.start()

            t1 = time.perf_counter()
            model.fit(X_train, y_train)
            t2 = time.perf_counter()

            _, train_peak = tracemalloc.get_traced_memory()
            tracemalloc.reset_peak()

            model.predict(X_test)
            t3 = time.perf_counter()
            _, test_peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()

            mem_data[i,] = (
                train_peak / (1024 * 1024),
                test_peak / (1024 * 1024)
            )

        for i in tqdm(range(self.n), total=self.n, desc=f"{name} (TIME)"):
            model = model_class(**kwargs)

            X_train, X_test, y_train, y_test = split_transpose(
                self.X, self.y,
                test_size=self.test_sz,
                random_state=rng
            )

            t1 = time.perf_counter()
            model.fit(X_train, y_train)
            t2 = time.perf_counter()
            preds = model.predict(X_test)
            t3 = time.perf_counter()

            time_data[i,] = (
                (t2 - t1) * 1000,
                (t3 - t2) * 1000,
                (y_test.flatten() == preds.flatten()).mean()
            )

        self.data[name] = (time_data, mem_data)

    def summary(self, baseline=None):
        aux = []
        for name, (time_data, mem_data) in self.data.items():
            result = {
                'model': name,
                'train_mean_ms': time_data[:, 0].mean(),
                'train_std_ms': time_data[:, 0].std(),
                'test_mean_ms': time_data[:, 1].mean(),
                'test_std_ms': time_data[:, 1].std(),
                'mean_accuracy': time_data[:, 2].mean(),
                'train_mem_mean_mb': mem_data[:, 0].mean(),
                'train_mem_std_mb': mem_data[:, 0].std(),
                'test_mem_mean_mb': mem_data[:, 1].mean(),
                'test_mem_std_mb': mem_data[:, 1].std()
            }
            aux.append(result)
        df = pd.DataFrame(aux).set_index('model')

        if baseline is not None and baseline in self.data:
            df['train_speedup'] = df.loc[baseline, 'train_mean_ms'] / df['train_mean_ms']
            df['test_speedup'] = df.loc[baseline, 'test_mean_ms'] / df['test_mean_ms']
            df['train_mem_reduction'] = df.loc[baseline, 'train_mem_mean_mb'] / df['train_mem_mean_mb']
            df['test_mem_reduction'] = df.loc[baseline, 'test_mem_mean_mb'] / df['test_mem_mean_mb']
        return df

## Ejemplo

In [12]:
# levantamos el dataset Wine, que tiene 13 features y 178 observaciones en total
X_full, y_full = get_wine_dataset()

X_full.shape, y_full.shape

((178, 13), (178, 1))

In [13]:
# encodeamos a número las clases
y_full_encoded = label_encode(y_full)

y_full[:5], y_full_encoded[:5]

(array([['class_0'],
        ['class_0'],
        ['class_0'],
        ['class_0'],
        ['class_0']], dtype='<U7'),
 array([[0],
        [0],
        [0],
        [0],
        [0]]))

In [60]:
# generamos el benchmark
# observar que son valores muy bajos de runs para que corra rápido ahora
b = Benchmark(
    X_full, y_full_encoded,
    n_runs = 100,
    warmup = 20,
    mem_runs = 20,
    test_sz = 0.3,
    same_splits = False
)

Benching params:
Total runs: 140
Warmup runs: 20
Peak Memory usage runs: 20
Running time runs: 100
Train size rows (approx): 125
Test size rows (approx): 53
Test size fraction: 0.3


In [13]:
# bencheamos un par
to_bench = [QDA]

for model in to_bench:
    b.bench(model)

QDA (TIME): 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 271.82it/s]


In [14]:
# como es una clase, podemos seguir bencheando más después
b.bench(TensorizedQDA)

TensorizedQDA (TIME): 100%|█████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 505.94it/s]


In [15]:
# hacemos un summary
b.summary()

,train_mean_ms,train_std_ms,test_mean_ms,test_std_ms,mean_accuracy,train_mem_mean_mb,train_mem_std_mb,test_mem_mean_mb,test_mem_std_mb
model,,,,,,,,,
QDA,0.362195,0.115624,2.979891,0.449799,0.982407,0.018380,0.000724,0.008446,0.001775
TensorizedQDA,0.377502,0.087030,1.270504,0.166855,0.982593,0.018187,0.000661,0.012032,0.000070


In [16]:
# son muchos datos! nos quedamos con un par nomás
summ = b.summary()

# como es un pandas DataFrame, subseteamos columnas fácil
summ[['train_mean_ms', 'test_mean_ms','mean_accuracy']]

,train_mean_ms,test_mean_ms,mean_accuracy
model,,,
QDA,0.362195,2.979891,0.982407
TensorizedQDA,0.377502,1.270504,0.982593


In [17]:
# podemos setear un baseline para que fabrique columnas de comparación
summ = b.summary(baseline='QDA')

summ

,train_mean_ms,train_std_ms,test_mean_ms,test_std_ms,mean_accuracy,train_mem_mean_mb,train_mem_std_mb,test_mem_mean_mb,test_mem_std_mb,train_speedup,test_speedup,train_mem_reduction,test_mem_reduction
model,,,,,,,,,,,,,
QDA,0.362195,0.115624,2.979891,0.449799,0.982407,0.018380,0.000724,0.008446,0.001775,1.000000,1.000000,1.00000,1.000000
TensorizedQDA,0.377502,0.087030,1.270504,0.166855,0.982593,0.018187,0.000661,0.012032,0.000070,0.959452,2.345441,1.01064,0.701904


In [18]:
summ[[
    'train_mean_ms', 'test_mean_ms','mean_accuracy',
    'train_speedup', 'test_speedup',
    'train_mem_reduction', 'test_mem_reduction'
]]

,train_mean_ms,test_mean_ms,mean_accuracy,train_speedup,test_speedup,train_mem_reduction,test_mem_reduction
model,,,,,,,
QDA,0.362195,2.979891,0.982407,1.000000,1.000000,1.00000,1.000000
TensorizedQDA,0.377502,1.270504,0.982593,0.959452,2.345441,1.01064,0.701904


# Consigna QDA

**Notación**: en general notamos

* $k$ la cantidad de clases
* $n$ la cantidad de observaciones
* $p$ la cantidad de features/variables/predictores

**Sugerencia:** combinaciones adecuadas de `transpose`, `stack`, `reshape` y, ocasionalmente, `flatten` y `diagonal` suele ser más que suficiente. Se recomienda **fuertemente* explorar la dimensionalidad de cada elemento antes de implementar las clases.

---

## Tensorización

En esta sección nos vamos a ocupar de hacer que el modelo sea más rápido para generar predicciones, observando que incurre en un doble `for` dado que predice en forma individual un escalar para cada observación, para cada clase. Paralelizar ambos vía tensorización suena como una gran vía de mejora de tiempos.

### 1) Diferencias entre `QDA`y `TensorizedQDA`

1. ¿Sobre qué paraleliza `TensorizedQDA`? ¿Sobre las $k$ clases, las $n$ observaciones a predecir, o ambas?

`QDA` hace un doble ciclo: primero sobre las observaciones y luego sobre las clases.

```python
for observación x_i:
    for clase j:
        calcular distancia de x_i a media de clase j usando matriz de covarianza j
        usar distancia para obtener verosimilitud
```

Para cada observacion $i$ y cada clase $j$:
- Resta la media de la clase: $(x_i - \mu_j)$
- Aplica la fórmula de distancia de Mahalanobis: $(x_i - \mu_j)^T \Sigma_j^{-1} (x_i - \mu_j)$
- Calcula una verosimilitud por clase

En cambio, `TensorizedQDA` itera sobre cada observación pero no para cada clase. Vectoriza sobre las clases usando tensores (`np.stack`), evitando el segundo loop.

```python
for observación x_i:
    calcular las verosimilitudes para todas las clases a la vez
```

Para cada observación $i$:
- Calcula las distancias a todas las medias de clase simultáneamente
- Evalúa la forma cuadrática para cada clase en paralelo
- Usa broadcasting para aplicar todas las $\Sigma_j^{-1}$ sin necesidad de bucles

`TensorizedQDA` paraleliza **sobre las $k$ clases**.

Esto se logra al apilar las medias y matrices de covarianza inversas por clase en tensores (`self.tensor_means` y `self.tensor_inv_covs`), lo que permite calcular la log-verosimilitud de una observación respecto a todas las clases simultáneamente utilizando broadcasting.

La paralelización ocurre en la linea: `unbiased_x = x - self.tensor_means`. Donde $x$ es una observación de dimensión $(p, 1)$ y `self.tensor_means` es una matriz de shape $(k, p, 1)$. Esta resta se vectoriza automáticamente sobre las k clases.


2. Analizar los shapes de `tensor_inv_covs` y `tensor_means` y explicar paso a paso cómo es que `TensorizedQDA` llega a predecir lo mismo que `QDA`.

- `self.tensor_means`: Tiene shape $(k, p, 1)$, contiene las medias $\mu_j$ para cada una de las $k$ clases, en dimensión $p$.  --> Cada clase tiene un vector columna que representa la media de cada feature.
- `self.tensor_inv_cov`: Tiene shape $(k, p, p)$, contiene las matrices inversas de covarianza $\Sigma_j^{-1}$ para cada clase. --> Cada clase tiene una matriz cuadrada de tamaño $(p \times p)$ que representa la inversa de su matriz de covarianza.

#### Paso a Paso:
1) Stackear los parámetros por clase
    - En `_fit_params(X, y)` se calcula la media (`self.means`) y la inversa de la matriz de covarianza (`self.inv_covs`) por clase.
    - Luego se apilan en tensores:
        - `self.tensor_means`: shape `(k, p, 1)`
        - `self.tensor_inv_cov`: shape `(k, p, p)` 
2) Restar todas las medias al mismo tiempo: `unbiased_x = x - self.tensor_means`
    - Se calcula la diferencia entre la observacion $x$ y cada media $\mu_j$
    - Usando *broadcasting*, esta operación se vectoriza automáticamente sobre las $k$ clases.
3)  Transformar en vectores fila: `unbiased_x.transpose(0, 2, 1)`
    - Transforma los vectores columna en vectores fila. 
    - Los prepara para que esten en formato correcto para el producto matricial de Mahalanobis.
4) Calcular la forma cuadratica vectorizada: `inner_prod = unbiased_x.transpose(0, 2, 1) @ self.tensor_inv_cov @ unbiased_x`
    - Obtener un escalar para cada clase con la medida de distancia para cada $j$
5) Calcular las log-verosimilitudes de cada clase: `return 0.5 * np.log(LA.det(self.tensor_inv_cov)) - 0.5 * inner_prod.flatten()`
    - Mahalanobis: `inner_prod.flatten()`
    - Se calcula la log-verosimilitud completa para cada clase simultaneamente.

El resultado es el mismo que en QDA porque ambos implementan la misma regla de decisión bayesiana. La diferencia esta en que `QDA`lo hace con un *for* sobre cada clase y `TensorizedQDA` lo hace en paralelo para todas las clases mediante tensores y broadcasting. 

### 2) Optimización

Debido a la forma cuadrática de QDA, no se puede predecir para $n$ observaciones en una sola pasada (utilizar $X \in \mathbb{R}^{p \times n}$ en vez de $x \in \mathbb{R}^p$) sin pasar por una matriz de $n \times n$ en donde se computan todas las interacciones entre observaciones. Se puede acceder al resultado recuperando sólo la diagonal de dicha matriz, pero resulta ineficiente en tiempo y (especialmente) en memoria. Aún así, es *posible* que el modelo funcione más rápido.

3. Implementar el modelo `FasterQDA` (se recomienda heredarlo de `TensorizedQDA`) de manera de eliminar el ciclo for en el método predict.

In [14]:
class FasterQDA(TensorizedQDA):
    def _predict_log_conditional(self, X):
        """
        X: shape (p, n) — cada columna es una observación
        self.tensor_means: shape (k, p, 1)
        self.tensor_inv_cov: shape (k, p, p)
        """
        p, n = X.shape
        k = self.tensor_means.shape[0]

        # Expandimos X: (p, n) → (1, p, n)
        X_exp = X[None, :, :]                         # (1, p, n)
        means = self.tensor_means                     # (k, p, 1)
        diffs = X_exp - means                         # (k, p, n)

        # Multiplicamos: (k, p, n) → (k, n, p)
        diffs_T = np.transpose(diffs, axes=(0, 2, 1))  # (k, n, p)

        # inv_cov: (k, p, p)
        # Primero: A = (k, n, p) @ (k, p, p) → (k, n, p)
        A = np.matmul(diffs_T, self.tensor_inv_cov)    # (k, n, p)

        # Segundo: M_full = A @ diffs_T.transpose(0,2,1) → (k, n, n)
        M_full = np.matmul(A, diffs)                   # (k, n, n)

        # Diagonal: (k, n)
        dists = np.array([np.diag(M_full[j]) for j in range(k)])  # (k, n)

        # Log-verosimilitud
        log_det = np.log(np.linalg.det(self.tensor_inv_cov))      # (k,)
        log_likelihood = 0.5 * log_det[:, None] - 0.5 * dists      # (k, n)
        return log_likelihood

    def predict(self, X):
        log_cond = self._predict_log_conditional(X)             # (k, n)
        posterior = log_cond + self.log_a_priori[:, None]       # (k, n)
        return np.argmax(posterior, axis=0).reshape(1, -1)      # (1, n)

#### Diferencias entre `TensorizedQDA` y `FasterQDA`

|                       | `TensorizedQDA`                           | `FasterQDA`                                      |
|-------------------------------|-------------------------------------------|--------------------------------------------------|
| **Paralelización**             | Solo sobre las clases $k$             | Sobre las clases $k$ y las observaciones $n$ |
| **Método `predict()`**         | Usa un `for` sobre observaciones (hereda de `BaseBayesianClassifier`). Llama a `_predict_one()` para cada observación | Reescribe `predict()` para vectorizar sobre todas las observaciones.  Calcula todo junto en `_predict_log_conditional(X)` |
| **Velocidad de predicción**   | Más lenta al hacer loop sobre $n$             | Más rapida al tener la vectorización completa                    |


`FasterQDA`:
- Elimina el bucle `for` del método `predict()`
- Vectoriza el cálculo de log-verosimilitud para todas las observaciones de una sola vez
- Aumenta la eficiencia

In [61]:
# pongo un par mas de corridas 
b = Benchmark(
    X_full, y_full_encoded,
    n_runs = 2000,
    warmup = 100,
    mem_runs = 2000,
    test_sz = 0.3,
    same_splits = False
)

Benching params:
Total runs: 4100
Warmup runs: 100
Peak Memory usage runs: 2000
Running time runs: 2000
Train size rows (approx): 125
Test size rows (approx): 53
Test size fraction: 0.3


In [62]:
models = [QDA, TensorizedQDA, FasterQDA]
for model in models:
    b.bench(model)

FasterQDA (TIME): 100%|██████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:01<00:00, 1517.86it/s]


In [63]:
summ = b.summary(baseline='TensorizedQDA')

summ[[
    'train_mean_ms', 'test_mean_ms','mean_accuracy', 'train_mem_mean_mb', 'test_mem_mean_mb',
    'train_speedup', 'test_speedup',
    'train_mem_reduction', 'test_mem_reduction'
]]

,train_mean_ms,test_mean_ms,mean_accuracy,train_mem_mean_mb,test_mem_mean_mb,train_speedup,test_speedup,train_mem_reduction,test_mem_reduction
model,,,,,,,,,
QDA,0.352374,3.064899,0.984472,0.017990,0.007570,1.002036,0.411374,0.997944,1.588236
TensorizedQDA,0.353092,1.260820,0.983861,0.017953,0.012023,1.000000,1.000000,1.000000,1.000000
FasterQDA,0.330305,0.082165,0.983722,0.017909,0.115292,1.068988,15.344959,1.002470,0.104286


Conclusiones:
- `FasterQDA` mantiene o mejora la precisión con respecto a `TensorizedQDA`.
- Su tiempo de predicción es muchísimo más rápido ($\Delta ~16.57$ veces más rápido), gracias a la vectorización completa sobre todas las observaciones.
- Consume más memoria en test ($\Delta ~9.53$ veces más memoria), debido a la construcción de matrices de forma $n \times n$ al calcular las formas cuadráticas de Mahalanobis para todas las clases y observaciones.

4. Mostrar dónde aparece la mencionada matriz de $n \times n$, donde $n$ es la cantidad de observaciones a predecir.

La matriz de $n \times n$ aparece al calcular la distancia de Mahalanobis entre todas las observaciones $x_i$ y $x_k$, respecto a cada clase $j$.

En el modelo `FasterQDA`, se computa la siguiente expresión:

`M_full = np.matmul(A, diffs)`

Que es equivalente a computar para cada clase $j$:

$$
M_j = (X - \mu_j)^T \Sigma_j^{-1} (X - \mu_j)
$$

donde:
- $X \in \mathbb{R}^{p \times n}$: contiene las $n$ observaciones como columnas,
- $\mu_j \in \mathbb{R}^p$: media de la clase $j$,
- $\Sigma_j^{-1} \in \mathbb{R}^{p \times p}$: matriz inversa de covarianza de la clase $j$,
- $M_j \in \mathbb{R}^{n \times n}$: matriz simétrica que contiene en su entrada $(i, k)$ el valor:

$$
M_j[i, k] = (x_i - \mu_j)^T \Sigma_j^{-1} (x_k - \mu_j)
$$


5. Demostrar que
$$
diag(A \cdot B) = \sum_{cols} A \odot B^T = np.sum(A \odot B^T, axis=1)
$$ 

es decir, que se puede "esquivar" la matriz de $n \times n$ usando matrices de $n \times p$. También se puede usar, de forma equivalente,

$$
np.sum(A^T \odot B, axis=0).T
$$

queda a preferencia del alumno cuál usar.

*falta escribirlo en términos matemáticos*

el elemento (i,i) de A·B es la suma de los productos de la fila i de A por la columna i de B. para evitar construir la matriz de n*n, formamos C con C(i,j) = A(i,j)·B(i,j) entonces cada fila i de C contiene justo esos terminos. al sumar cada fila de C obtenemos directamente los valores de la diagonal de A·B


6. Utilizar la propiedad antes demostrada para reimplementar la predicción del modelo `FasterQDA` de forma eficiente en un nuevo modelo `EfficientQDA`.

In [64]:
class EfficientQDA(QDA):
    def _predict_log_conditional(self, X):
        num_clases = len(self.inv_covs)
        num_observaciones = X.shape[1]
        log_condicional = np.zeros((num_clases, num_observaciones))
        log_det_inv = [np.log(np.linalg.det(cov_inv)) for cov_inv in self.inv_covs]
        for clase in range(num_clases):
            diferencias = X - self.means[clase]
            proyeccion = self.inv_covs[clase].dot(diferencias)
            log_condicional[clase] = 0.5 * log_det_inv[clase] - 0.5 * (diferencias * proyeccion).sum(axis=0)
        return log_condicional

    def predict(self, X):
        log_condicional = self._predict_log_conditional(X)
        posterior = log_condicional + self.log_a_priori[:, None]
        return np.argmax(posterior, axis=0).reshape(1, -1)



7. Comparar la performance de las 4 variantes de QDA implementadas hasta ahora (no Cholesky) ¿Qué se observa? A modo de opinión ¿Se condice con lo esperado?

In [65]:
models.append(EfficientQDA)
b.bench(EfficientQDA)
summ = b.summary(baseline='QDA')
summ[['train_mean_ms','test_mean_ms','mean_accuracy','train_mem_mean_mb','test_mem_mean_mb','train_speedup','test_speedup']]

EfficientQDA (TIME): 100%|███████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:01<00:00, 1548.45it/s]


,train_mean_ms,test_mean_ms,mean_accuracy,train_mem_mean_mb,test_mem_mean_mb,train_speedup,test_speedup
model,,,,,,,
QDA,0.352374,3.064899,0.984472,0.017990,0.007570,1.000000,1.000000
TensorizedQDA,0.353092,1.260820,0.983861,0.017953,0.012023,0.997968,2.430877
FasterQDA,0.330305,0.082165,0.983722,0.017909,0.115292,1.066816,37.301706
EfficientQDA,0.312722,0.090980,0.983537,0.017918,0.029634,1.126797,33.687630


In [66]:
summ[["train_speedup", "test_speedup", "train_mem_reduction", "test_mem_reduction", "mean_accuracy"]]

,train_speedup,test_speedup,train_mem_reduction,test_mem_reduction,mean_accuracy
model,,,,,
QDA,1.000000,1.000000,1.000000,1.000000,0.984472
TensorizedQDA,0.997968,2.430877,1.002060,0.629629,0.983861
FasterQDA,1.066816,37.301706,1.004535,0.065661,0.983722
EfficientQDA,1.126797,33.687630,1.003982,0.255455,0.983537


TensorizedQDA triplica la velocidad con un ligero aumento de RAM; FasterQDA acelera ~35× pero dispara el consumo de memoria; EfficientQDA logra ese mismo ~35× sin el pico de RAM. Estos resultados confirman la estrategia esperada: tensorizar, vectorizar y aplicar la fórmula de la diagonal para un QDA rápido y eficiente en memoria.

---
## Cholesky

Hasta ahora todos los esfuerzos fueron enfocados en realizar una predicción más rápida. Los tiempos de entrenamiento (teóricos al menos) siguen siendo los mismos o hasta (minúsculamente) peores, dado que todas las mejoras siguen llamando al método `_fit_params` original de `QDA`.

La descomposición/factorización de [Cholesky](https://en.wikipedia.org/wiki/Cholesky_decomposition#Statement) permite factorizar una matriz definida positiva $A = LL^T$ donde $L$ es una matriz triangular inferior. En particular, si bien se asume que $p \ll n$, invertir la matriz de covarianzas $\Sigma$ para cada clase impone un cuello de botella que podría alivianarse. Teniendo en cuenta que las matrices de covarianza son simétricas y salvo degeneración, definidas positivas, Cholesky como mínimo debería permitir invertir la matriz más rápido.

*Nota: observar que calcular* $A^{-1}b$ *equivale a resolver el sistema* $Ax=b$.

### 3) Diferencias entre implementaciones de `QDA_Chol`


8. Si una matriz $A$ tiene fact. de Cholesky $A=LL^T$, expresar $A^{-1}$ en términos de $L$. ¿Cómo podría esto ser útil en la forma cuadrática de QDA?

Si A = LL^T, entonces A⁻¹ = (L⁻¹)^T L⁻¹. Esto es útil en QDA porque:

1. La forma cuadrática en QDA es: (x-μ)ᵀ Σ⁻¹ (x-μ)
2. Si Σ = LLᵀ, entonces Σ⁻¹ = (L⁻¹)ᵀ L⁻¹
3. Podemos reescribir la forma cuadrática como:
   (x-μ)ᵀ Σ⁻¹ (x-μ) = (x-μ)ᵀ (L⁻¹)ᵀ L⁻¹ (x-μ) = (L⁻¹(x-μ))ᵀ (L⁻¹(x-μ)) = ||L⁻¹(x-μ)||²

Esto permite:
- Evitar calcular explícitamente Σ⁻¹
- Calcular la forma cuadrática resolviendo sistemas triangulares (más eficiente)
- Mejor estabilidad numérica


9. Explicar las diferencias entre `QDA_Chol1`y `QDA` y cómo `QDA_Chol1` llega, paso a paso, hasta las predicciones.

Diferencias entre QDA_Chol1 y QDA:

1. En lugar de almacenar Σ⁻¹ (inversa de covarianza), QDA_Chol1 almacena L⁻¹ (inversa del factor de Cholesky)
2. El cálculo de la probabilidad logarítmica se modifica para usar la factorización

Pasos de QDA_Chol1:
1. Durante el fit:
   - Calcula la descomposición de Cholesky: Σ = LLᵀ
   - Almacena L⁻¹ (inversa de L) para cada clase
   - Calcula y almacena las medias μ para cada clase

2. Durante la predicción (para cada punto x y clase):
   - Calcula x centrado: x_centered = x - μ
   - Aplica L⁻¹: y = L⁻¹ @ x_centered
   - Calcula log_prob = log(det(Σ⁻¹)) - yᵀy
     Donde:
     - log(det(Σ⁻¹)) = 2*sum(log(diag(L⁻¹))) (por propiedades de Cholesky)
     - yᵀy es la norma al cuadrado de y


10. ¿Cuáles son las diferencias entre `QDA_Chol1`, `QDA_Chol2` y `QDA_Chol3`?

Las tres implementaciones usan Cholesky pero difieren en:

1. QDA_Chol1:
   - Calcula L⁻¹ explícitamente usando scipy.linalg.inv()
   - Almacena L⁻¹
   - Multiplica directamente L⁻¹ @ x_centered

2. QDA_Chol2:
   - Almacena L (no su inversa)
   - Usa solve_triangular para resolver L y = x_centered (equivalente a y = L⁻¹x_centered pero más eficiente)
   - log(det(Σ⁻¹)) se calcula como -2*sum(log(diag(L)))

3. QDA_Chol3:
   - Similar a QDA_Chol1 pero calcula L⁻¹ usando dtrtri de LAPACK (más eficiente para matrices triangulares)
   - dtrtri es una rutina optimizada específicamente para invertir matrices triangulares

En general:
- QDA_Chol2 es la más eficiente (evita calcular inversas explícitas)
- QDA_Chol3 es mejor que QDA_Chol1 pero peor que QDA_Chol2
- QDA_Chol1 es la menos eficiente (inversión explícita)


11. Comparar la performance de las 7 variantes de QDA implementadas hasta ahora ¿Qué se observa?¿Hay alguna de las implementaciones de `QDA_Chol` que sea claramente mejor que las demás?¿Alguna que sea peor?

In [67]:
chol_models = [ QDA_Chol1, QDA_Chol2, QDA_Chol3]
models  += chol_models

for model in chol_models:
    b.bench(model)

summ = b.summary(baseline='QDA')
summ[['train_mean_ms', 'test_mean_ms', 'mean_accuracy']]

QDA_Chol3 (TIME): 100%|███████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:04<00:00, 470.76it/s]


,train_mean_ms,test_mean_ms,mean_accuracy
model,,,
QDA,0.352374,3.064899,0.984472
TensorizedQDA,0.353092,1.260820,0.983861
FasterQDA,0.330305,0.082165,0.983722
EfficientQDA,0.312722,0.090980,0.983537
QDA_Chol1,0.404313,1.847268,0.983741
QDA_Chol2,0.321017,4.061022,0.984120
QDA_Chol3,0.289177,1.591216,0.983676


In [68]:
summ[["train_speedup", "test_speedup", "train_mem_reduction", "test_mem_reduction", "mean_accuracy"]]

,train_speedup,test_speedup,train_mem_reduction,test_mem_reduction,mean_accuracy
model,,,,,
QDA,1.000000,1.000000,1.000000,1.000000,0.984472
TensorizedQDA,0.997968,2.430877,1.002060,0.629629,0.983861
FasterQDA,1.066816,37.301706,1.004535,0.065661,0.983722
EfficientQDA,1.126797,33.687630,1.003982,0.255455,0.983537
QDA_Chol1,0.871539,1.659152,1.002100,0.975604,0.983741
QDA_Chol2,1.097682,0.754711,1.009441,0.992358,0.984120
QDA_Chol3,1.218544,1.926136,1.008721,0.994437,0.983676


¿Qué se observa en general?

    Consistencia en Precisión

        Todas las variantes mantienen la misma precisión que el QDA original, demostrando que las optimizaciones no comprometen la calidad predictiva.

    Rendimiento en Velocidad

        Predicción:

            Las versiones vectorizadas (FasterQDA y EfficientQDA) destacan por su velocidad significativamente mayor, gracias a la paralelización sobre observaciones y clases.
            TensorizedQDA también supera al QDA estándar al paralelizar solo sobre clases.
            Entre las implementaciones Cholesky, QDA_Chol3 es la única que supera ligeramente al QDA original.

        Entrenamiento:

            EfficientQDA y QDA_Chol2 son los más rápidos.
            QDA_Chol1 y QDA_Chol3 tienen un costo adicional por el cálculo de inversas.

    Eficiencia en Memoria

        FasterQDA consume más memoria durante la predicción debido a matrices temporales.
        Las versiones Cholesky (QDA_Chol1/2/3) son las más eficientes en memoria.

¿Alguna es claramente mejor?

Entre las QDA_Chol, en términos de velocidad QDA_Chol1 y QDA_Chol3 tienen un rendimiento similar y considerablemente superior a QDA_Chol2. En términos técnicos, QDA_Chol2 es superior debido a que evita calcular inversas explícitas usando solve_triangular, lo que mejora la estabilidad numérica (especialmente con matrices casi singulares) y reduce errores de redondeo en cálculos intermedios.

¿Alguna es peor?

En términos de velocidad QDA_Chol2 es considerablemente mas lenta. A los efectos prácticos resulta la peor en las condiciones de prueba.


### 4) Optimización


12. Implementar el modelo `TensorizedChol` paralelizando sobre clases/observaciones según corresponda. Se recomienda heredarlo de alguna de las implementaciones de `QDA_Chol`, aunque la elección de cuál de ellas queda a cargo del alumno según lo observado en los benchmarks de puntos anteriores.

Como se mencionó anteriormente, si bien la segunda implementación de cholesky (sin calcular la inversa) es mas estable numericamente, resulta la mas lenta de las tres. Nuestros benchmarks se centran en eficiencia en termino de velocidad (speedup) y memoria (mem_reduction), dado que la precisión (mean_accuracy) es similar en todas las implementaciones. Como además no hay una diferencia en la eficiencia en memoria, tomamos alguna de las más rápidas para tensorizar.

In [69]:
class TensorizedQDA_Chol1(QDA_Chol1):
    def _fit_params(self, X, y):
        super()._fit_params(X, y)

        # Stack porque queremos vectorizar sobre las clases
        self.tensor_L_inv = np.stack(self.L_invs)        # (k, p, p)
        self.tensor_means = np.stack(self.means)         # (k, p, 1)

    def _predict_log_conditionals(self, x):
        # x: (p, 1)
        unbiased_x = x - self.tensor_means               # (k, p, 1)
        y = self.tensor_L_inv @ unbiased_x               # (k, p, 1)
        
        # usamos axis1 y 2 en la diagonal porque necesitamos diagonal para cada clase (matriz de pxp)
        log_det = np.log(np.prod(np.diagonal(self.tensor_L_inv, axis1=1, axis2=2), axis=1))  # (k,)
        quad_term = 0.5 * (y ** 2).sum(axis=1).flatten()  # (k,)

        return log_det - quad_term

    def _predict_one(self, x):
        return np.argmax(self.log_a_priori + self._predict_log_conditionals(x))


class TensorizedQDA_Chol3(QDA_Chol3):
    def _fit_params(self, X, y):
        super()._fit_params(X, y)

        self.tensor_L_invs = np.stack(self.L_invs)    # shape (k, p, p)
        self.tensor_means = np.stack(self.means)      # shape (k, p, 1)

    def _predict_log_conditionals(self, x):

        unbiased_x = x - self.tensor_means            # shape (k, p, 1)
        #print(unbiased_x.shape)
        y = self.tensor_L_invs @ unbiased_x           # shape (k, p, 1)

        quad_term = 0.5* (y**2).sum(axis=1).flatten()           # shape (k,)
        # usamos axis1 y 2 en la diagonal porque necesitamos diagonal para cada clase (matriz de pxp)
        log_det = np.log(self.tensor_L_invs.diagonal(axis1=1, axis2=2)).sum(axis=1)  # shape (k,)

        return log_det - 0.5 * quad_term

    def _predict_one(self, x):
        return np.argmax(self.log_a_priori + self._predict_log_conditionals(x))


In [70]:
# un benchmark auxiliar para testear los cholesky tensorizados
b_aux = Benchmark(
    X_full, y_full_encoded,
    n_runs = 500,
    warmup = 50,
    mem_runs = 500,
    test_sz = 0.3,
    same_splits = False
)

Benching params:
Total runs: 1050
Warmup runs: 50
Peak Memory usage runs: 500
Running time runs: 500
Train size rows (approx): 125
Test size rows (approx): 53
Test size fraction: 0.3


In [71]:
implementations = [QDA, TensorizedQDA_Chol1, TensorizedQDA_Chol3]

for model in implementations:
    b_aux.bench(model)


TensorizedQDA_Chol3 (TIME): 100%|███████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 649.58it/s]


In [72]:

summ = b_aux.summary(baseline="QDA")
summ[["train_speedup", "test_speedup", "train_mem_reduction", "test_mem_reduction", "mean_accuracy"]]

,train_speedup,test_speedup,train_mem_reduction,test_mem_reduction,mean_accuracy
model,,,,,
QDA,1.000000,1.000000,1.000000,1.000000,0.984778
TensorizedQDA_Chol1,0.804154,2.510083,1.001283,0.592228,0.983037
TensorizedQDA_Chol3,0.999520,2.678317,1.012306,0.600606,0.982259


A efectos de tomar una decision sobre en cual basar el EfficientChol, miramos los benchmarks para Chol1 y Chol3.
No se aprecian diferencias significativas. Tomamos Chol3 solo porque presenta una media de speedup superior a Chol1.


13. Implementar el modelo `EfficientChol` combinando los insights de `EfficientQDA` y `TensorizedChol`. Si se desea, se puede implementar `FasterChol` como ayuda, pero no se contempla para el punto.

Comenzamos por implementar el FastChol, eliminando el loop de `_predict_one`, llamando una sola vez al método `predict`. Esto es, vectorizamos sobre las observaciones ademas de sobre las clases. De esta forma se espera eliminar el overhead de `python` de ese loop (que es el que mas iteraciones realiza). Por lo que se espera una ganancia en tiempo muy superior a la obtenida al tensorizar solo sobre las clases


In [73]:
# heredamos de chol3 porque vimos que practiamente no hay diferencia de rendimiento entre cholesky 1 y 3
class FasterQDA_Chol3(TensorizedQDA_Chol3):
    def _predict_log_conditional(self, X):

        p, n = X.shape
        k = self.tensor_means.shape[0]

        # broadcast
        X_exp = X[None, :, :]                          # (1, p, n)
        means = self.tensor_means                      # (k, p, 1)
        diffs = X_exp - means                          # (k, p, n)

        y = np.matmul(self.tensor_L_invs, diffs)       # (k, p, n)

        quad_term = (y ** 2).sum(axis=1)                    # (k, n)

        # log(prod diag L_inv) = sum log diag
        log_det = np.log(self.tensor_L_invs.diagonal(axis1=1, axis2=2)).sum(axis=1)  # (k,)

        # Final log-likelihoods
        log_likelihoods = log_det[:, None] - 0.5 * quad_term  # (k, n)
        return log_likelihoods

    def predict(self, X):

        log_cond = self._predict_log_conditional(X)             # (k, n)
        posterior = log_cond + self.log_a_priori[:, None]       # (k, n)
        return np.argmax(posterior, axis=0).reshape(1, -1)      # (1, n)

In [74]:
# heredamos de faster, tienen el mismo predict
class EfficientChol3(FasterQDA_Chol3):
    def _fit_params(self, X, y):
        k = self.log_a_priori.shape[0]
        p, n = X.shape
        self.tensor_Linv = np.zeros((k, p, p))
        self.tensor_means = np.zeros((k, p, 1))

        for idx in range(k):
            class_X = X[:, y.flatten() == idx]
            cov = np.cov(class_X, bias=True)
            L = cholesky(cov, lower=True)
            L_inv = dtrtri(L, lower=1)[0]  # L^-1
            self.tensor_Linv[idx] = L_inv
            self.tensor_means[idx] = class_X.mean(axis=1, keepdims=True)

    def _predict_log_conditional(self, X):
       
        p, n = X.shape
        k = self.tensor_Linv.shape[0]

        X_exp = X[None, :, :]                      # (1, p, n)
        means = self.tensor_means                  # (k, p, 1)
        diffs = X_exp - means                      # (k, p, n)

        Y = np.matmul(self.tensor_Linv, diffs)     # (k, p, n)

        squared_norms = np.sum(Y**2, axis=1)       # (k, n)
        log_det = np.log(np.prod(np.abs(self.tensor_Linv.diagonal(axis1=1, axis2=2)), axis=1))  # (k,)

        log_likelihood = log_det[:, None] - 0.5 * squared_norms
        return log_likelihood


In [75]:
# agregamos los dos modelos al benchmark auxiliar,
# solo para ver que esta funcionando
b_aux.bench(EfficientChol3)

EfficientChol3 (TIME): 100%|███████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1785.02it/s]


In [76]:

summ = b_aux.summary(baseline="QDA")
summ[["train_speedup", "test_speedup", "train_mem_reduction", "test_mem_reduction", "mean_accuracy"]]

,train_speedup,test_speedup,train_mem_reduction,test_mem_reduction,mean_accuracy
model,,,,,
QDA,1.000000,1.000000,1.000000,1.000000,0.984778
TensorizedQDA_Chol1,0.804154,2.510083,1.001283,0.592228,0.983037
TensorizedQDA_Chol3,0.999520,2.678317,1.012306,0.600606,0.982259
EfficientChol3,1.135122,48.118461,0.725027,0.106114,0.984519


Un comportamiento consistente con la tensorización para el QDA regular (sin cholesky). En este caso las mejoras en velocidad son
aun mayores.
Se puede ver que si bien almacenar las matrices para la doble tensorización implica un mayor uso de memoria (`test_mem_reduction=0.1`), la ganancia en tiempos de ejcución es mayor incluso a las obtenidas para el método eficiente de QDA base.


14. Comparar la performance de las 9 variantes de QDA implementadas ¿Qué se observa? A modo de opinión ¿Se condice con lo esperado?

In [77]:
# Agregamos los nuevos modelos al benchmark principal
chol_tensorized_models = [TensorizedQDA_Chol3,  EfficientChol3]
models += chol_tensorized_models # solo para registro
for model in chol_tensorized_models:
    b.bench(model)

EfficientChol3 (TIME): 100%|█████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:01<00:00, 1730.90it/s]


In [78]:
summ = b.summary(baseline='QDA')

summ[["train_speedup", "test_speedup", "train_mem_reduction", "test_mem_reduction", "mean_accuracy"]]

,train_speedup,test_speedup,train_mem_reduction,test_mem_reduction,mean_accuracy
model,,,,,
QDA,1.000000,1.000000,1.000000,1.000000,0.984472
TensorizedQDA,0.997968,2.430877,1.002060,0.629629,0.983861
FasterQDA,1.066816,37.301706,1.004535,0.065661,0.983722
EfficientQDA,1.126797,33.687630,1.003982,0.255455,0.983537
QDA_Chol1,0.871539,1.659152,1.002100,0.975604,0.983741
QDA_Chol2,1.097682,0.754711,1.009441,0.992358,0.984120
QDA_Chol3,1.218544,1.926136,1.008721,0.994437,0.983676
TensorizedQDA_Chol3,1.110294,3.111211,1.011917,0.599993,0.982380
EfficientChol3,1.248386,54.459457,0.723115,0.106026,0.983972



En primer lugar, como se destacó con los modelos anteriores, las tensorizaciones basadas en la descomposición de Cholesky no comprometen la precisión con respecto al QDA base.

En general, se observa que la principal vía para mejorar el rendimiento de los modelos es eliminar bucles explícitos en Python y reemplazarlos por operaciones vectorizadas con NumPy. En particular, los mayores saltos en eficiencia se logran al aplicar **doble tensorización**: primero sobre el conjunto de clases y luego sobre las observaciones. Esta última es especialmente relevante debido a que en la mayoría de los problemas, el número de observaciones (**n**) es mucho mayor que el número de clases (**k**), por lo que evitar un bucle sobre observaciones tiene un impacto notable.

Esto se refleja en los incrementos de performance observados: pasar de `TensorizedQDA` (que sólo vectoriza sobre clases) a `FasterQDA` o `EfficientQDA` (que también tensorizan sobre observaciones) implica un **salto de velocidad de aproximadamente x30** frente al modelo `QDA` original.

Finalmente, al utilizar la versión `EfficientChol3`, se obtiene un **speedup cercano a x45** con respecto al `QDA` base. Esta mejora se debe a dos factores clave:

1. Se mantiene la doble tensorización (sobre clases y observaciones).
2. Se reemplaza la inversión directa de matrices de covarianza por una descomposición de Cholesky, seguida de una inversión eficiente de la matriz triangular inferior (dtrtri).
En conjunto, estas optimizaciones explican los beneficios sustanciales observados en términos de tiempo de ejecución, sin afectar la precisión del modelo.

Por último, el costo que hay que pagar para mejorar tensorizando es un mayor uso de memoria, como se observa en la degradación de los resultados para `test_mem_reduction` de las versiones tensorizadas.